In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
train=pd.read_csv("C:/Users/Richard/Datasets/DonorsChoose/train.csv")
test=pd.read_csv("C:/Users/Richard/Datasets/DonorsChoose/test.csv")
resources=pd.read_csv("C:/Users/Richard/Datasets/DonorsChoose/resources.csv")

C:\Users\Richard\AppData\Local\conda\conda\envs\data_science\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## create these features for each proposal:
- total price
- highest single item price

In [3]:
resources['total_price']=resources['price']*resources['quantity'];
                    #if as_index=True by default, these are series; otherwise, it returns a DataFrame
#rg2=resources.groupby(by='id')['total_price'].agg('sum')#same
rg1=resources.groupby(by='id', as_index=False)['total_price'].sum();
rg2=resources.groupby(by='id',as_index=False)['price'].max();
rg2.rename({'price':'highest_single_item_price'},axis='columns',inplace=True);#rename column name
rg12=pd.merge(rg1,rg2,how='inner',on='id');

## Join the tables got from resources.csv & train.csv; resources.csv & test.csv

In [4]:
train1=pd.merge(train,rg12,how='left',on='id')
test1=pd.merge(test,rg12,how='left',on='id')

train1['project_submitted_datetime']=pd.to_datetime(train1['project_submitted_datetime'])
train1['project_submitted_datetime']=train1['project_submitted_datetime'].dt.date
test1['project_submitted_datetime']=pd.to_datetime(test1['project_submitted_datetime'])
test1['project_submitted_datetime']=test1['project_submitted_datetime'].dt.date
y_train1=train1['project_is_approved']
X_train1=train1.drop(['project_is_approved'],axis=1)
X_test1=test1

## reasons not to use sub categories

In [5]:
set(X_test1['project_subject_subcategories'].unique())^set(X_train1['project_subject_subcategories'].unique())

{'Applied Sciences, Warmth, Care & Hunger',
 'Civics & Government, Extracurricular',
 'Civics & Government, Music',
 'Civics & Government, Nutrition Education',
 'Civics & Government, Other',
 'Civics & Government, Performing Arts',
 'Civics & Government, Team Sports',
 'Civics & Government, Warmth, Care & Hunger',
 'Community Service, ESL',
 'Community Service, Nutrition Education',
 'Community Service, Team Sports',
 'ESL, Economics',
 'ESL, Team Sports',
 'ESL, Warmth, Care & Hunger',
 'Economics, Foreign Languages',
 'Economics, Health & Life Science',
 'Economics, Music',
 'Economics, Other',
 'Economics, Team Sports',
 'Extracurricular, Foreign Languages',
 'Extracurricular, Nutrition Education',
 'Extracurricular, Social Sciences',
 'Financial Literacy, Performing Arts',
 'Foreign Languages, Gym & Fitness',
 'Foreign Languages, Team Sports',
 'Gym & Fitness, Parent Involvement',
 'Gym & Fitness, Warmth, Care & Hunger',
 'Health & Life Science, Warmth, Care & Hunger',
 'History &

In [6]:
X_test1['project_subject_subcategories'].nunique()

390

## extract complete categorical values for creating one-hot-encoding

In [7]:
complete_prefix=list(X_train1['teacher_prefix'].dropna(axis=0, how='any').unique())
complete_state=list(X_train1['school_state'].unique())
complete_grade=list(X_train1['project_grade_category'].unique())
complete_category=list(X_train1['project_subject_categories'].unique())#51 categories
#complete_sub_category=list(X_train1['project_subject_subcategories'].unique())#407 sub-categories


In [8]:
from sklearn.base import BaseEstimator, TransformerMixin

class add_features(BaseEstimator, TransformerMixin):
    """Takes in dataframe, extracts road name column, outputs average word length"""

    def __init__(self):
        pass #self.vars = vars  # e.g. pass in a column name to extract

    def convert_log(self, previous_proposals):#passed in is a Series?no datapoint
        #return np.mean([len(word) for word in name.split()])
        result=np.log(1+float(previous_proposals))
        #print(result)
        return result

    def transform(self, df, y=None):
        """The workhorse of this feature extractor"""
        """features about reject pressure. project viewers need to reject certain percentage
        of proposals due to limited resources. 
        This is probably a better way to use datetime info"""
        pd.options.mode.chained_assignment = None#ignore SettingWithCopyWarning because it's false positive, everything I do here is on a copy and I return a copy. if I change the names of df to df1,df2 etc, the warning should be gone.

        df.loc[:,'reject_pressure_by_funds']=df.groupby('project_submitted_datetime')['total_price'].transform(lambda x: x.sum())
        df.loc[:,'reject_pressure_by_projects']=df.groupby('project_submitted_datetime')['total_price'].transform(lambda x: x.count())
        df.loc[:,'log_previously_posted_projects']=df['teacher_number_of_previously_posted_projects'].apply(self.convert_log)
        #create DataFrames that contain categorical information
        prefix = pd.get_dummies(df['teacher_prefix'],drop_first=False)
        state=pd.get_dummies(df['school_state'],drop_first=False)
        grade= pd.get_dummies(df['project_grade_category'],drop_first=False)
        category= pd.get_dummies(df['project_subject_categories'],drop_first=False)
        #sub_category=pd.get_dummies(df['project_subject_subcategories'],drop_first=False)
        missing_cols_main=set(complete_prefix+complete_state+complete_grade+complete_category)-(set(list(prefix))|set(list(state))|set(list(grade))|set(list(category)))
        #missing_cols_extra=set(complete_sub_category)-set(list(sub_category))
        #set(complete_category)&set(complete_sub_category) not empty
        #print (missing_cols_main)
        #print (missing_cols_extra)
        df=pd.concat([df,prefix],axis=1)
        df=pd.concat([df,state],axis=1)
        df=pd.concat([df,grade],axis=1)
        df=pd.concat([df,category],axis=1)
        #df=pd.concat([df,sub_category],axis=1)
        for c1 in missing_cols_main:
            df.loc[:,c1] = 0
        #for c2 in missing_cols_extra:
        #    df.loc[:,c2] = 0
        #del prefix,state,grade,category#,sub_category
        #del missing_cols_main
        #del missing_cols_extra

        to_be_dropped_sure=['id',
         'teacher_id',
         'teacher_prefix',
         'school_state',
         'project_submitted_datetime',
         'project_grade_category',
         'project_subject_categories',
         'project_subject_subcategories',
         'project_title',
         'project_essay_1',
         'project_essay_2',
         'project_essay_3',
         'project_essay_4',
         'project_resource_summary',
         'teacher_number_of_previously_posted_projects']
        to_be_dropped=to_be_dropped_sure#+complete_prefix+complete_state+complete_grade+complete_category#+complete_sub_category
        #print(df.drop(to_be_dropped,axis=1).head(1))
        df.drop(to_be_dropped,axis=1,inplace=True)
        return df

    def fit(self, df, y=None,**fit_params):
        """Returns `self` unless something different happens in train and test"""
        return self

In [9]:
import sklearn
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.linear_model import LinearRegression,Lasso
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import cross_val_score,StratifiedKFold,ShuffleSplit
import xgboost as xgb

C:\Users\Richard\AppData\Local\conda\conda\envs\data_science\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [10]:
xgb_learner=xgb.XGBRegressor #eg. xgb_learner(max_depth=3, n_estimators=300, learning_rate=0.05) in the pipeline
simplest_learner=LinearRegression
lasso=Lasso
steps=[('features', FeatureUnion([
    ('add_features', add_features()) # can pass in either a pipeline or a transformer#
    ])),('scaler',StandardScaler()),('estimator',xgb_learner())]
pipeline=Pipeline(steps)#

In [11]:
""" Modelling using Xgboost """
'''parameters by Ehsan 'subsample': 0.8, 
              'colsample_bytree': 0.75,
              #'min_child_weight' : 1.5,
              'scale_pos_weight': 0.1796871962992238,
              'objective': 'binary:logistic', 
              'lambda': 1.5,
              'alpha': .6'''
from sklearn.model_selection import train_test_split



X_train2, X_valid2, y_train2, y_valid2 = train_test_split(StandardScaler().fit_transform(add_features().transform(X_train1)), y_train1, test_size=0.30, random_state=238234)
params = {'eta': 0.1, 
          'max_depth': 6, 
          'objective': 'binary:logistic', 
          'min_child_weight' : 6,
          'eval_metric': 'auc',
          'seed': 2856,
          #'silent': True,
          'colsample':0.8}


In [12]:
type(X_train1),type(X_train2)

(pandas.core.frame.DataFrame, numpy.ndarray)

after train_test_split Dataframe->ndarray now

In [13]:
watchlist = [(xgb.DMatrix(X_train2, y_train2), 'train'), (xgb.DMatrix(X_valid2, y_valid2), 'valid')]
model = xgb.train(params, xgb.DMatrix(X_train2, y_train2), 1000,  watchlist, verbose_eval=10, early_stopping_rounds=20)



[0]	train-auc:0.61423	valid-auc:0.607894
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 20 rounds.
[10]	train-auc:0.637873	valid-auc:0.622925
[20]	train-auc:0.643867	valid-auc:0.626459
[30]	train-auc:0.651866	valid-auc:0.629526
[40]	train-auc:0.661038	valid-auc:0.631853
[50]	train-auc:0.66816	valid-auc:0.633329
[60]	train-auc:0.674749	valid-auc:0.634756
[70]	train-auc:0.679244	valid-auc:0.635466
[80]	train-auc:0.684926	valid-auc:0.636321
[90]	train-auc:0.689219	valid-auc:0.636754
[100]	train-auc:0.69293	valid-auc:0.6368
[110]	train-auc:0.69755	valid-auc:0.637249
[120]	train-auc:0.700636	valid-auc:0.636842
[130]	train-auc:0.705116	valid-auc:0.637305
[140]	train-auc:0.708729	valid-auc:0.638037
[150]	train-auc:0.711966	valid-auc:0.638295
[160]	train-auc:0.714999	valid-auc:0.638392
[170]	train-auc:0.718973	valid-auc:0.638477
[180]	train-auc:0.722081	valid-auc:0.638415
[190]	train-auc:0.725085	valid-auc:0.6

In [14]:
#print('best score is {}'.format(best_learner.best_score_))
#predictions=best_learner.predict(test1)

In [17]:
predictions = model.predict(xgb.DMatrix(StandardScaler().fit_transform(add_features().transform(X_test1))), ntree_limit=model.best_ntree_limit)
my_submission = pd.DataFrame({'id': X_test1['id'], 'project_is_approved': predictions})
my_submission.to_csv('xgb.csv', index=False)

In [18]:
my_submission

,id,project_is_approved
0,p233245,0.913966
1,p096795,0.889631
2,p236235,0.668938
3,p233680,0.780085
4,p171879,0.783047
5,p016071,0.848914
6,p099906,0.852982
7,p200236,0.825959
8,p129452,0.890824
9,p186652,0.953506
